# Chinese Eateries Analysis in U.S.
Date: Sept 23, 2021

Prepared by ***Keqi Deng***

In [16]:
import numpy as np
import os
import pandas as pd

## Data Source
The data used in this analysis is provided by [Yelp Opendata](https://www.yelp.com/dataset), last update on Jan 21, 2021.

In [17]:
yelp_df = pd.read_json('yelp_dataset/yelp_academic_dataset_business.json', lines=True)
yelp_df.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."


In [18]:
# Dataset information
yelp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160585 entries, 0 to 160584
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   160585 non-null  object 
 1   name          160585 non-null  object 
 2   address       160585 non-null  object 
 3   city          160585 non-null  object 
 4   state         160585 non-null  object 
 5   postal_code   160585 non-null  object 
 6   latitude      160585 non-null  float64
 7   longitude     160585 non-null  float64
 8   stars         160585 non-null  float64
 9   review_count  160585 non-null  int64  
 10  is_open       160585 non-null  int64  
 11  attributes    145593 non-null  object 
 12  categories    160470 non-null  object 
 13  hours         133244 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 17.2+ MB


In [21]:
# Dataset length:
len(yelp_df)

160585

## Dataset Modification
Modify the Dataset to Find Restaurants Labeled *Chinese*

In [25]:
chinese_df = yelp_df[yelp_df.categories.str.contains('Chinese', na=False)]
chinese_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
53,djolOChjDtxniurUFP_SXA,Panda Express,1460 Rinehart Rd,Sanford,FL,32771,28.800686,-81.331712,2.5,47,1,"{'RestaurantsGoodForGroups': 'True', 'GoodForK...","Restaurants, Chinese, Fast Food","{'Monday': '10:30-21:0', 'Tuesday': '10:30-21:..."
75,3ME_CSB1bo4F0QMhQRUeOA,Yan's China Bistro,146 Humphrey St,Swampscott,MA,01907,42.468081,-70.916752,4.0,74,1,"{'RestaurantsGoodForGroups': 'True', 'NoiseLev...","Restaurants, Chinese","{'Monday': '11:30-22:0', 'Tuesday': '11:30-22:..."
101,bS5DScsLQSFDcbGnPD2GbQ,Sage Clinic,487 Davie Street,Vancouver,BC,V6B 2G2,49.275872,-123.123857,3.0,8,0,"{'ByAppointmentOnly': 'False', 'WiFi': 'u'free...","Acupuncture, Naturopathic/Holistic, Health & M...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-19:0', ..."
105,TbZDLpBOl-EbO2LfMySrEg,First China Restaurant,5295 Buford Hwy NE,Atlanta,GA,30340,33.896640,-84.280918,3.0,11,0,"{'RestaurantsGoodForGroups': 'True', 'Business...","Restaurants, Chinese",None
132,5eq56X-e9YbAKmHgUOyXSg,Hunan Lion,2038 Crown Plaza,Columbus,OH,43220,40.065378,-83.071415,4.0,162,1,"{'GoodForKids': 'True', 'RestaurantsGoodForGro...","Chinese, Restaurants","{'Monday': '11:30-22:0', 'Tuesday': '11:30-22:..."
